In [1]:
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

In [2]:
# Extract data for Q2, already done. The data is saved to:
# https://raw.githubusercontent.com/Jeff-HOU/bpi.2018/master/data/data_q2.csv
# ?token=AilEXVc0O5KlZviHuRGCT7A4Luhovbleks5a8BUjwA%3D%3D

# df = pd.read_csv('F:\\Competitions\\BPI\\data\\data.csv')

# # df_q2 = df[['case', 'event', 'year', 'startTime', 'applicant', 'department', 'doctype', 'subprocess', 'number_parcels',
# #            'young farmer', 'small farmer', 'area', 'redistribution',
# #            'penalty_B3', 'penalty_B4', 'penalty_B5', 'penalty_B5F', 'penalty_B6', 'penalty_B16', 'penalty_BGK', 'penalty_BGKV',
# #            'penalty_C16', 'penalty_JLP3', 'penalty_V5',
# #            'amount_applied0', 'amount_applied1', 'amount_applied2', 'amount_applied3',
# #            'penalty_amount0', 'penalty_amount1', 'penalty_amount2', 'penalty_amount3',
# #            'payment_actual0', 'payment_actual1', 'payment_actual2', 'payment_actual3', 'rejected']]

# df_q2 = df[['case', 'department', 'number_parcels', 'young farmer', 'small farmer', 'area', 'redistribution',
#            'penalty_B3', 'penalty_B4', 'penalty_B5', 'penalty_B5F', 'penalty_B6', 'penalty_B16', 'penalty_BGK', 'penalty_BGKV',
#            'penalty_C16', 'penalty_JLP3', 'penalty_V5', 'rejected']]

# df_q2 = df_q2.drop_duplicates(subset='case')
# df_q2 = df_q2.fillna(0)
# df_q2['severe'] = df_q2['penalty_B3'] | df_q2['penalty_B4'] | df_q2['penalty_B5'] | df_q2['penalty_B5F'] | df_q2['penalty_B6'] | df_q2['penalty_B16'] | df_q2['penalty_BGK'] | df_q2['penalty_BGKV'] | df_q2['penalty_C16'] | df_q2['penalty_JLP3'] | df_q2['penalty_V5']
# df_q2.to_csv('F:\\Competitions\\BPI\\data\\data_q2.csv')

# del df, df_q2
# import gc
# gc.collect()

In [3]:
# The csv file is large, please download it first then read from local files
# if you insist in reading online, read it from the link above

df_q2 = pd.read_csv('F:\\Competitions\\BPI\\data\\data_q2.csv')

df_q2_core = df_q2[['severe', 'department', 'number_parcels', 'young farmer', 'small farmer', 'area', 'redistribution']]

In [4]:
bools = df_q2_core.select_dtypes(include='bool').columns.tolist()
bool_to_int = df_q2_core.loc[:, bools].astype(int)
cat_to_dummies = pd.get_dummies(df_q2_core['department'], prefix='dept').iloc[:, 1:]
df_q2_core = df_q2_core.drop(columns=bools)
df_q2_core = df_q2_core.drop(columns='department')
df_q2_core = pd.concat([df_q2_core, bool_to_int, cat_to_dummies], axis=1)
df_q2_core.columns = df_q2_core.columns.str.replace('\s+', '_')

In [5]:
with pd.option_context('display.max_columns', None):
    display(df_q2_core.head())
#     display(df_q2_core[df_q2_core['severe'] == True][:5])

,number_parcels,area,severe,young_farmer,small_farmer,redistribution,dept_6b,dept_d4,dept_e7
0,3,2.6994,0,0,1,1,0,0,1
1,16,93.5478,0,0,0,1,0,1,0
2,11,19.4835,0,0,0,1,0,0,1
3,5,19.4835,0,0,0,1,0,0,0
4,20,133.6535,0,1,0,1,0,1,0


In [6]:
X_0 = df_q2_core.columns.tolist()
X_0.remove('severe')
X = df_q2_core[X_0]
y = df_q2_core['severe']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
train = pd.concat([y_train, X_train], axis=1)
lm_0 = smf.ols(formula='severe ~ ' + ' + '.join(X_0),
               data=train).fit()
lm_0.summary().tables[1]

lm_0_summary = pd.DataFrame({'coef':lm_0.params, 'pvalues': lm_0.pvalues})
lm_0_sig = lm_0_summary.where(lm_0.pvalues < 0.05).dropna()
print('The linear model including all trace attributes:\n', lm_0_summary)
print('The significant coefficients:\n', lm_0_sig)
print('\nThe adjusted R-squared is',lm_0.rsquared_adj)

X_1 = [col for col in lm_0_sig.index.tolist() if col is not 'Intercept']
lm_1 = smf.ols(formula='severe ~ ' + ' + '.join(X_1),
               data=train).fit()
lm_1_summary = pd.DataFrame({'coef':lm_1.params, 'pvalues': lm_1.pvalues})
print('\nBuild a new linear model including only the significant coefficients from last model:\n', lm_1_summary)
print('\nThe adjusted R-squared is', lm_1.rsquared_adj)
print("It's better than the last one." if abs(lm_1.rsquared_adj)>abs(lm_1.rsquared_adj) else "It's worse than the last one")

The linear model including all trace attributes:
                     coef       pvalues
Intercept       0.086471  3.007519e-04
number_parcels  0.000310  7.544279e-03
area           -0.000042  9.106380e-02
young_farmer    0.003736  3.317347e-01
small_farmer    0.008393  9.974511e-02
redistribution -0.060662  1.116453e-02
dept_6b         0.018834  2.096408e-11
dept_d4         0.020510  3.211551e-09
dept_e7         0.001218  6.492177e-01
The significant coefficients:
                     coef       pvalues
Intercept       0.086471  3.007519e-04
number_parcels  0.000310  7.544279e-03
redistribution -0.060662  1.116453e-02
dept_6b         0.018834  2.096408e-11
dept_d4         0.020510  3.211551e-09

The adjusted R-squared is 0.0024406638787262125

Build a new linear model including only the significant coefficients from last model:
                     coef       pvalues
Intercept       0.088656  2.058038e-04
number_parcels  0.000128  2.688836e-02
redistribution -0.060812  1.096060e-02
de